In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import math

In [21]:
df=pd.read_csv(r"C:\Users\anant\OneDrive\Documents\GitHub\3rd-year-resources-2022-scheme-rvce\6th sem\AI363IA-Natural Language processing and transformers(NLP)\LAB\Musical_instruments_reviews 4.csv")
df.dropna(subset=['reviewText'], inplace=True)
X = df['summary']
y = df.iloc[:, 5]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=42,stratify=y)
df


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5.0,good,1393545600,"02 28, 2014"
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5.0,Jake,1363392000,"03 16, 2013"
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",The primary job of this device is to block the...,5.0,It Does The Job Well,1377648000,"08 28, 2013"
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",Nice windscreen protects my MXL mic and preven...,5.0,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014"
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",This pop filter is great. It looks and perform...,5.0,No more pops when I record my vocals.,1392940800,"02 21, 2014"
...,...,...,...,...,...,...,...,...,...
10256,A14B2YH83ZXMPP,B00JBIVXGC,Lonnie M. Adams,"[0, 0]","Great, just as expected. Thank to all.",5.0,Five Stars,1405814400,"07 20, 2014"
10257,A1RPTVW5VEOSI,B00JBIVXGC,Michael J. Edelman,"[0, 0]",I've been thinking about trying the Nanoweb st...,5.0,"Long life, and for some players, a good econom...",1404259200,"07 2, 2014"
10258,AWCJ12KBO5VII,B00JBIVXGC,Michael L. Knapp,"[0, 0]",I have tried coated strings in the past ( incl...,4.0,Good for coated.,1405987200,"07 22, 2014"
10259,A2Z7S8B5U4PAKJ,B00JBIVXGC,"Rick Langdon ""Scriptor""","[0, 0]","Well, MADE by Elixir and DEVELOPED with Taylor...",4.0,Taylor Made,1404172800,"07 1, 2014"


In [22]:
df['summary'].isna().sum()


np.int64(0)

In [23]:
X_train

6580                                                great
7657                                          works great
7070                                  Just what you need.
6054                                    I give it 4 stars
8182                                            The best!
                              ...                        
8495                                  Just as I expected.
6010                                   Great simple cable
2076    D'Adarrio EXL140-3D Nickel Wound Electric Guit...
6753    A tool to help me get my goal in becoming a Lu...
5433                                   Just what I needed
Name: summary, Length: 7177, dtype: object

In [24]:
X_test

6468                                  Nice starter Ukulele
6104                                            Perfect...
7306                           Necessary to a gold upgrade
9636                          I feel like a blues brother.
581      Works but had to "ruin" a strap for everything...
                               ...                        
7143                                           Music stand
2304                         GREAT!!! GOOD STURDY PIECE!!!
10021                Tons of Fun. Great with a double neck
755                       Buy these. They are really good.
4616                              It's the right tool.....
Name: summary, Length: 3077, dtype: object

In [25]:
def preprocess(text):
    stop_words=set(stopwords.words('english'))
    # text=text.lower()
    words=word_tokenize(text)
    list1=[]

    for word in words:
        if word not in stop_words and word.isalnum():
            list1.append(word)
    
    list_doc=' '.join(list1)
    return list_doc

In [12]:
X_train=X_train.apply(preprocess)
X_test=X_test.apply(preprocess)
# X_processed = X.apply(preprocess)
type(X_train)


pandas.core.series.Series

In [ ]:
def dff(word,sentence):
    words=sentence.split()
    return words.count(word)/(len(words)+1)

def idff(word,sentences):
    n=0
    for sent in sentences:
        if word in sent:
            n+=1
    return math.log(len(sentences)/(n+1))

all_docs=X_train.to_list()+X_test.to_list()


unique_words=set(' '.join(all_docs).split())

idf_score={}
for word in unique_words:
    idf_score[word]=idff(word,all_docs)

#vocabulary
vocab=list(idf_score.keys())

X_train_vector=[]
for sentence in X_train:
    temp_list=[]
    for w in vocab:
        tfidf_score=dff(w,sentence)*idf_score[w]
        temp_list.append(tfidf_score)
    X_train_vector.append(temp_list)


X_test_vector = []
for sentence in X_test:
    temp_list = []
    for w in vocab:
        tfidf_score = dff(w, sentence)*idf_score[w]
        temp_list.append(tfidf_score)
    X_test_vector.append(temp_list)

In [27]:
all_docs

['great',
 'works great',
 'Just need',
 'I give 4 stars',
 'The best',
 'Works Great',
 'Great vibe pedal',
 'pleased',
 'Excellent guitar hanger',
 'Works great',
 'I like Planet Waves pick key chain',
 'lovely strings',
 'Not kid proof',
 'Not lot protection',
 'Thank You',
 'Great strap acoustic guitars',
 'works gr4eat',
 'But Not Great',
 'Great guitar money',
 'XLR Female To Male Adapter',
 'Bronze Strings',
 'Incredible Deal',
 'An Essential Compressor That I Prefer Over Boss',
 'Nothing extraordinary',
 'Will put Ditto Looper business',
 'Get em',
 'Not bright',
 'This',
 'Awesome power',
 'Fender Footswitch',
 'Sound Device',
 'ok',
 'Does NOT work acoustic electric guitars',
 'Jazz Picks great',
 'nice quality price right',
 'Exactly musician needs',
 'Great Strap',
 'Just need',
 'Solid All Around',
 'Great Manufacturer',
 'The go distortion pedal',
 'Not Sure I know use',
 'Great Product',
 'Nice Heavy Distortion',
 '3 pin male inmale adapter',
 'best results good amp',
 '

In [26]:
unique_words

{'payed',
 'Lookiin',
 'smelling',
 'Problems',
 'Convenioence',
 'arround',
 'Traveling',
 'tabletop',
 'PA130MM',
 'starter',
 'stock',
 'unpowered',
 'Flexible',
 'tested',
 '8001912',
 'models',
 'bang',
 'Casual',
 'mine',
 'Podcasting',
 'Drywall',
 'Tension',
 'Goldilocks',
 'improvement',
 'value',
 'Proof',
 'Vamped',
 'Iconic',
 'MB1',
 'countless',
 'Blow',
 'Instrument',
 'strangs',
 'true',
 'pperfect',
 'Muffler',
 'II',
 'lock',
 'horrible',
 'Arm',
 'Locking',
 '514ce',
 'Weight',
 'Product',
 'Rotosounds',
 'tips',
 'Led',
 'sand',
 'least',
 'Cylindrical',
 'phosphor',
 'Space',
 'Hot',
 '34',
 'knots',
 'joke',
 'Duet',
 'Wax',
 'oz',
 'Cleaning',
 'lemony',
 'Skinny',
 'firmware',
 'larger',
 'Perfectly',
 'Dreamy',
 'Well',
 'dull',
 'always',
 'Exceptional',
 'Cost',
 'death',
 'aid',
 'stringwinder',
 'shoulder',
 'blend',
 'Promising',
 'Old',
 'breezy',
 'Drove',
 'similar',
 'sure',
 'boom',
 'packed',
 'sent',
 'EG20D',
 'Shape',
 'LED',
 'waxes',
 'par',
 'D

In [9]:
X_train_vector=np.array(X_train_vector)
X_test_vector=np.array(X_test_vector)

In [10]:
model=LogisticRegression(max_iter=100)
model.fit(X_train_vector,y_train)

LogisticRegression()

In [11]:
text = "serious musicians look elsewhere"

text=preprocess(text)
vector=[]

for word in vocab:
    tfs=dff(word,text)
    tf_idf_score=tfs*idf_score[word]
    vector.append(tf_idf_score)

output=model.predict([vector])
output


array([3.])